In [27]:
import numpy as np
from scipy.spatial.distance import cdist
import random

# Step 1: Read the TSP file using the open() method
def read_tsp_file(filename):
    coordinates = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith(('EOF', 'NAME', 'COMMENT', 'TYPE', 'DIMENSION', 'EDGE_WEIGHT_TYPE')):
                continue
            parts = line.strip().split()
            if len(parts) == 3:  # It's a coordinate line
                node_id, x, y = map(float, parts)
                coordinates.append((x, y))
    return np.array(coordinates)

# Step 2: Calculate the distance matrix
def calculate_distance_matrix(coords):
    return cdist(coords, coords, metric='euclidean')

# Step 3: Generate a path using the Nearest Neighbor heuristic
def generate_nearest_neighbor_path(dist_matrix):
    num_nodes = len(dist_matrix)
    unvisited = set(range(num_nodes))  # All cities are unvisited initially
    start_city = random.randint(0, num_nodes - 1)  # Select a random starting city
    path = [start_city]  # Start with the randomly chosen city
    unvisited.remove(start_city)

    current_city = start_city
    while unvisited:
        # Find the nearest unvisited city
        nearest_city = min(unvisited, key=lambda city: dist_matrix[current_city][city])
        path.append(nearest_city)
        unvisited.remove(nearest_city)
        current_city = nearest_city

    return path

# # Optional: Commented out the random shuffle function
# def generate_random_path(num_nodes):
#     path = list(range(num_nodes))
#     random.shuffle(path)
#     return path

# Step 4: Calculate the total distance of the path, ensuring the salesperson returns to the starting city
def calculate_total_distance(path, dist_matrix):
    total_distance = 0.0
    for i in range(len(path)):
        total_distance += dist_matrix[path[i-1]][path[i]]
    # Add the distance from the last city back to the first city
    total_distance += dist_matrix[path[-1]][path[0]]
    return total_distance

# Execution
# File path is set to "C:\Users\pc\Desktop\TPTO\instances TSP\berlin52.tsp"
filename = r'C:\Users\pc\Desktop\TPTO\instances TSP\berlin52.tsp'

coordinates = read_tsp_file(filename)
distance_matrix = calculate_distance_matrix(coordinates)

# Use the Nearest Neighbor heuristic to generate a path with a random starting city
nearest_neighbor_path = generate_nearest_neighbor_path(distance_matrix)

# Print the Nearest Neighbor path
print(f'Nearest Neighbor path (node indices): {nearest_neighbor_path}')

# Calculate the total distance of the Nearest Neighbor path
total_distance = calculate_total_distance(nearest_neighbor_path, distance_matrix)

print(f'Total distance of the Nearest Neighbor path: {total_distance}')


Nearest Neighbor path (node indices): [20, 30, 17, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 43, 15, 49, 19, 22, 29, 28, 46, 25, 26, 27, 11, 50, 10, 51, 12, 13, 42, 9, 8, 7, 40, 18, 44, 2, 16, 41, 6, 1, 32]
Total distance of the Nearest Neighbor path: 11397.335020435998


In [17]:
import numpy as np
import random
from scipy.spatial.distance import cdist

# Utility Functions
def read_tsp_file(filename):
    coordinates = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith(('EOF', 'NAME', 'COMMENT', 'TYPE', 'DIMENSION', 'EDGE_WEIGHT_TYPE')):
                continue
            parts = line.strip().split()
            if len(parts) == 3:  # It's a coordinate line
                _, x, y = map(float, parts)
                coordinates.append((x, y))
    return np.array(coordinates)

def calculate_distance_matrix(coordinates):
    return cdist(coordinates, coordinates, metric='euclidean')

def calculate_total_distance(path, dist_matrix):
    total_distance = 0.0
    for i in range(len(path)):
        total_distance += dist_matrix[path[i - 1]][path[i]]
    return total_distance

def calculate_fitness(path, dist_matrix):
    total_distance = calculate_total_distance(path, dist_matrix)
    return 1 / total_distance  # Lower distance means higher fitness

# Genetic Algorithm Components
def generate_initial_population(pop_size, num_cities, dist_matrix):
    population = []

    # Nearest Neighbor Initialization
    def nearest_neighbor():
        unvisited = list(range(num_cities))
        start_city = random.choice(unvisited)
        path = [start_city]
        unvisited.remove(start_city)
        while unvisited:
            last_visited = path[-1]
            nearest_city = min(unvisited, key=lambda city: dist_matrix[last_visited][city])
            path.append(nearest_city)
            unvisited.remove(nearest_city)
        return path

    # Half Nearest Neighbor, Half Random
    for _ in range(pop_size // 2):
        population.append(nearest_neighbor())
    for _ in range(pop_size // 2):
        path = list(range(num_cities))
        random.shuffle(path)
        population.append(path)

    return population

def tournament_selection(population, fitnesses, tournament_size=3):
    selected = random.sample(list(zip(population, fitnesses)), tournament_size)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

def order_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size

    # Randomly select a subarray from parent1
    start, end = sorted(random.sample(range(size), 2))
    child[start:end] = parent1[start:end]

    # Fill the rest from parent2, maintaining order
    idx = end
    for gene in parent2:
        if gene not in child:
            if idx >= size:
                idx = 0
            child[idx] = gene
            idx += 1

    return child

def mutate(path, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(path)), 2)
        path[i], path[j] = path[j], path[i]
    return path

def local_search_2opt(path, dist_matrix):
    size = len(path)
    improved = True
    while improved:
        improved = False
        for i in range(size - 1):
            for j in range(i + 2, size):  # Avoid reversing a single city
                if j - i == 1:
                    continue
                new_path = path[:i] + path[i:j][::-1] + path[j:]
                if calculate_total_distance(new_path, dist_matrix) < calculate_total_distance(path, dist_matrix):
                    path = new_path
                    improved = True
    return path

def adaptive_mutation_rate(initial_rate, current_gen, max_gen):
    return max(0.01, initial_rate * (1 - current_gen / max_gen))

# Main Genetic Algorithm
def genetic_algorithm_tsp(dist_matrix, pop_size, generations, initial_mutation_rate, elite_size=2, target_distance=7544):
    num_cities = len(dist_matrix)
    population = generate_initial_population(pop_size, num_cities, dist_matrix)
    best_path = None
    best_fitness = -float('inf')
    reached_target = False
    previous_best_distance = float('inf')

    for gen in range(generations):
        # Calculate fitness for all individuals
        fitnesses = [calculate_fitness(ind, dist_matrix) for ind in population]
        max_fitness = max(fitnesses)
        current_best_path = population[np.argmax(fitnesses)]
        best_distance = calculate_total_distance(current_best_path, dist_matrix)

        # Ensure we only keep improving (never going up)
        if best_distance < previous_best_distance:
            previous_best_distance = best_distance
            best_fitness = max_fitness
            best_path = current_best_path

        # Elite selection
        elites = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)[:elite_size]
        elites = [e[0] for e in elites]

        # Generate new population
        new_population = elites[:]
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child = order_crossover(parent1, parent2)
            child = mutate(child, adaptive_mutation_rate(initial_mutation_rate, gen, generations))
            new_population.append(child)

        # Replace population
        population = new_population

        # Apply local search every 100 generations or at the end
        if gen % 100 == 0 or gen == generations - 1:
            best_path = local_search_2opt(best_path, dist_matrix)
            best_distance = calculate_total_distance(best_path, dist_matrix)
            print(f"Generation {gen + 1}: Best Distance = {best_distance:.2f}")

            # Check if the best distance meets the target
            if best_distance <= target_distance and not reached_target:
                print(f"Target distance {target_distance} reached at Generation {gen + 1}. Optimization stopped.")
                reached_target = True
                # Lock the best path, no further changes
                return best_path, best_distance

    # Final best distance after the loop (if not stopped early)
    best_distance = calculate_total_distance(best_path, dist_matrix)
    return best_path, best_distance

# Parameters
coordinates = read_tsp_file(r"C:\Users\pc\Desktop\TPTO\wiw\berlin52.tsp")
distance_matrix = calculate_distance_matrix(coordinates)

pop_size = 300
generations = 500
mutation_rate = 0.5
target_distance = 7544

# Execute
best_solution, best_distance = genetic_algorithm_tsp(distance_matrix, pop_size, generations, mutation_rate, target_distance=target_distance)
print("Best Total Distance:", best_distance)


Generation 1: Best Distance = 7864.94
Generation 101: Best Distance = 7598.44
Generation 201: Best Distance = 7598.44
Generation 301: Best Distance = 7598.44
Generation 401: Best Distance = 7598.44
Generation 500: Best Distance = 7598.44
Best Total Distance: 7598.4423409045385
